## Model Analysis w session OHE and Class Experts approach w GradientBoosting

In [1]:
#import sys
#sys.path.insert(0,"/Library/Python/2.7/site-packages/") 
#sys.path.insert(0,"/Library/Frameworks/Python.framework/Versions/2.7/bin")

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

In [2]:
data = pd.read_csv("./data/df_all_inc_sess.csv")

In [18]:
# get X and y
X = data[0:213466]
y = pd.read_csv("./data/y.csv")

In [19]:
# drop user_id
X = X.drop(['user_id'], axis=1)

In [20]:
# for users w/out session info, set relevant features to -1.
X.fillna(value=-1, inplace=True)

In [21]:
print X.shape
print y.shape

(213466, 764)
(213466, 1)


In [22]:
y.country_destination.unique()

array(['NDF', 'US', 'other', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL',
       'DE', 'AU'], dtype=object)

In [23]:
# encode y values SO THAT y has value 1 for the "single_class" and value 0 for all other classes..
y_labels = {}
single_class = "FR"
i = 1
for value in y.country_destination.unique():
    if value not in y_labels.keys():
        if value == single_class:
            y_labels[value] = 1
        else:
            y_labels[value] = 0
    if value == single_class:
        y[y == value] = 1
    else:
        y[y == value] = 0

print "y is indexed by dict:\n%s" % y_labels

y is indexed by dict:
{'FR': 1, 'NL': 0, 'PT': 0, 'CA': 0, 'DE': 0, 'IT': 0, 'US': 0, 'NDF': 0, 'other': 0, 'AU': 0, 'GB': 0, 'ES': 0}


In [24]:
# set y to array-like type
y = y.country_destination
y.ravel().astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
# train / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [26]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(149426, 764)
(149426,)
(64040, 764)
(64040,)


## Gradient Boosting
## for "NDF", n_estimators=100, max_depth=3 => Acc=0.7184 BUT, # correctly classified NDFs are 30335, it was 32051 when all classes were classified together!!!
## for "FR", n_estimators=100, max_depth=3 => Acc=0.9763

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, verbose=1)
gbc.fit(X_train, y_train.astype(int))
y_pred_gbc = gbc.predict(X_test)

      Iter       Train Loss   Remaining Time 
         1           0.2224           25.94m
         2           0.2214           27.91m
         3           0.2207           27.48m
         4           0.2200           26.57m
         5           0.2195           25.94m
         6           0.2190           25.34m
         7           0.2186           24.64m
         8           0.2181           24.24m
         9           0.2177           23.51m
        10           0.2173           23.12m
        20           0.2146           19.46m
        30           0.2125           16.31m
        40           0.2111           13.87m
        50           0.2101           11.38m
        60           0.2092            9.03m
        70           0.2081            6.77m
        80           0.2074            4.56m
        90           0.2068            2.28m
       100           0.2062            0.00s


In [28]:
print metrics.accuracy_score(y_test.astype(int), y_pred_gbc.astype(int))

0.976374141162


In [29]:
conf_matrix_gbc = metrics.confusion_matrix(y_test.astype(int), y_pred_gbc.astype(int))
print conf_matrix_gbc
print sum(sum(conf_matrix_gbc))

[[62524    21]
 [ 1492     3]]
64040
